<a href="https://colab.research.google.com/github/HughYau/Research-with-Python/blob/main/Crawler/Weibo_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import emoji
import time
import re
import xlwt
import xlrd
import pandas as pd
import numpy as np

In [ ]:
headers = {
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Mobile Safari/537.36 Edg/99.0.1150.39',
    'referer':'https://m.weibo.cn/status/LgUYJApRa?',
    'accept': 'application/json, text/plain, */*',
#    'cookie':'WEIBOCN_FROM=1110006030; loginScene=102003; SUB=_2A25PKc3ZDeRhGeFN61IY9inPyTiIHXVs1dORrDV6PUJbkdAKLVfHkW1NQJqiEQBfDqdtIn5Xx_7X2FM0fOAVuVHZ; _T_WM=64481388519; XSRF-TOKEN=6fc47a; MLOGIN=1; M_WEIBOCN_PARAMS=oid=4740322778679204&luicode=20000061&lfid=4740322778679204&uicode=20000061&fid=4740322778679204'
#     'cookie':'WEIBOCN_FROM=1110005030; loginScene=102003; SUB=_2A25PKtHIDeRhGeFN61IY9inPyTiIHXVs1P-ArDV6PUJbkdAKLWv5kW1NQJqiERUl2bOF3iqm6ygyibjCrOu2OXnG; _T_WM=34126505065; XSRF-TOKEN=47c223; MLOGIN=1; M_WEIBOCN_PARAMS=oid=4740322778679204&luicode=20000061&lfid=4740322778679204&uicode=20000061&fid=4740322778679204'
     'cookie':'XSRF-TOKEN=0cf293; WEIBOCN_FROM=1110005030; MLOGIN=0; loginScene=102003; M_WEIBOCN_PARAMS=oid=4740322778679204&luicode=20000061&lfid=4740322778679204; SUB=_2A25PKtHIDeRhGeFN61IY9inPyTiIHXVs1P-ArDV6PUJbkdAKLWv5kW1NQJqiERUl2bOF3iqm6ygyibjCrOu2OXnG'
}

In [ ]:
ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
def base62_decode(string, alphabet=ALPHABET):
    """Decode a Base X encoded string into the number
 
    Arguments:
    - `string`: The encoded string
    - `alphabet`: The alphabet to use for encoding
    """
    base = len(alphabet)
    strlen = len(string)
    num = 0
 
    idx = 0
    for char in string:
        power = (strlen - (idx + 1))
        num += alphabet.index(char) * (base ** power)
        idx += 1
 
    return num
def url_to_mid(url):
    '''
    >>> url_to_mid('z0JH2lOMb')
    3501756485200075L
    >>> url_to_mid('z0Ijpwgk7')
    3501703397689247L
    >>> url_to_mid('z0IgABdSn')
    3501701648871479L
    >>> url_to_mid('z08AUBmUe')
    3500330408906190L
    >>> url_to_mid('z06qL6b28')
    3500247231472384L
    >>> url_to_mid('yCtxn8IXR')
    3491700092079471L
    >>> url_to_mid('yAt1n2xRa')
    3486913690606804L
    '''
    url = str(url)[::-1]
    size = len(url) // 4 if len(url) % 4 == 0 else len(url) // 4 + 1
    result = []
    for i in range(size):
        s = url[i * 4: (i + 1) * 4][::-1]
        s = str(base62_decode(str(s)))
        s_len = len(s)
        if i < size - 1 and s_len < 7:
            s = (7 - s_len) * '0' + s
        result.append(s)
    result.reverse()
    return int(''.join(result))

In [ ]:
# 获取评论首页
def get_comment_index(mid,datas):
    url = f'https://m.weibo.cn/comments/hotflow?id={mid}&mid={mid}&max_id_type=0'
    response=requests.get(url,headers=headers).json()
    total_number = response.get('data').get('total_number')
    Xpath=response.get('data').get('data')
    for i in Xpath:
#         total_number=i.get('total_number')
#         id=i.get('id')
        data = {
            'user_name': i.get('user').get('screen_name'),
            'commtent': emoji.demojize(re.sub('<[^<]+?>', '', i.get('text'))),
            'data_time': i.get('created_at'),
            'mid': mid,
            'total_number' : total_number,
        }
        #保存数据
        datas.append(data)
    max_id = response.get('data').get('max_id')
    if max_id!=0:
        #获取多页评论
        time.sleep(5)
        get_comment(mid, max_id,datas)
# 获取多页评论

def get_comment(mid, max_id,datas):
    url = f'https://m.weibo.cn/comments/hotflow?id={mid}&mid={mid}&max_id={max_id}&max_id_type=0'
    response = requests.get(url, headers=headers).json()
    if response.get('data') is None:
        return 0
    else:
        total_number = response.get('data').get('total_number')
        Xpath=response.get('data').get('data')
    for i in Xpath:
#         total_number=i.get('total_number')
#         id=i.get('id')
        data = {
            'user_name': i.get('user').get('screen_name'),
            'commtent': emoji.demojize(re.sub('<[^<]+?>', '', i.get('text'))),
            'data_time': i.get('created_at'),
            'mid': mid,
            'total_number' : total_number,  
        }
        #保存数据
        datas.append(data)
    max_id = response.get('data').get('max_id')
    #获取多页评论
    if max_id!=0:
        time.sleep(5)
        get_comment(mid, max_id,datas)

In [ ]:
def turn_time(delta_days):
    """
    delta_days:就是那个五位数
    """
    real_time=pd.to_datetime('1899-12-30')+ pd.Timedelta(str(delta_days)+'D')#eg:'44701D'
    #上一步得到的就是一个pandas的时间戳形式，然后我们转换成合适的时间格式就好了
    #Timestamp('1899-12-30 00:00:00')
    return real_time.strftime("%Y-%m-%d")
turn_time(44616.0)

'2022-02-24'

In [ ]:
print(np.array([len(datas_russia),len(datas_ukraine) ,len(datas_randu) ,len(datas_mainland) ,len(datas_west)])/len(datas_randu))

[2.10798122 4.44225352 1.         1.57558685 1.47605634]


In [ ]:
2245+4731+1065+1678+1572

11291

In [ ]:
datas_russia = []
datas_ukraine = []
datas_randu = []
datas_mainland = []
datas_west = []

In [ ]:
datas_randu?

In [ ]:

workbook = xlrd.open_workbook('微博网址.xls')
table = workbook.sheet_by_name('Sheet1')
rows = table.nrows
for row in range(1,rows):
#     datas = []
    date = turn_time(table[row][0].value)
    subject = table[row][1].value
    media = table[row][2].value
    title = table[row][3].value
    url = table[row][4].value
    mid = str(url_to_mid(url))
    print(row,date,media,title,mid)
#     get_comment_index(mid,datas)
#     pd.DataFrame(datas).to_csv('.\\大陆\\'+date+'_'+subject+'_'+url+'.csv',encoding = 'utf_8_sig')


    if subject == '俄罗斯':
        get_comment_index(mid,datas_russia)
    elif subject == '乌克兰':
        get_comment_index(mid, datas_ukraine)
    elif subject == 'Russia-Ukriane':
        get_comment_index(mid, datas_randu)
    elif subject == '中国':
        get_comment_index(mid,datas_mainland)
    elif subject == '西方':
        get_comment_index(mid,datas_west)
    else:
        print('wrong')

    time.sleep(30)

# pd.DataFrame(datas_russia).to_csv('.\\大陆\\俄罗斯.csv',encoding = 'utf_8_sig')
# pd.DataFrame(datas_ukraine).to_csv('.\\大陆\\乌克兰.csv',encoding = 'utf_8_sig')
# pd.DataFrame(datas_randu).to_csv('.\\大陆\\俄罗斯和乌克兰.csv',encoding = 'utf_8_sig')
# pd.DataFrame(datas_mainland).to_csv('.\\大陆\\大陆.csv',encoding = 'utf_8_sig')
# pd.DataFrame(datas_west).to_csv('.\\大陆\\西方.csv',encoding = 'utf_8_sig')
        

1 2022-02-24 澎湃新闻 #俄乌局势最新进展#【正在直播｜乌克兰宣布与俄罗斯断交，全境进入战时状态】#乌总统发布全国总动员令#  4740322778679204
2 2022-02-24 澎湃新闻 【泽连斯基呼吁民众“挺身而出”，将向希望得到武器的人发武器】#乌克兰总统呼吁民众挺身而出# 4740467260922173
3 2022-02-25 澎湃新闻 【美国总统#拜登称将冻结俄罗斯在美所有资产#】#俄罗斯在美所有资产将被冻结# 4740666338574356
4 2022-02-25 澎湃新闻 #泽连斯基喊话欧洲领导人#：今天不帮乌克兰，明天就有战争上门】#乌总统呼吁欧洲国家施以援手# 4740677163814177
5 2022-02-25 澎湃新闻 【乌总统泽连斯基：西方已经完全放弃了乌克兰】#乌总统称西方完全放弃了乌克兰# 4740682151625891
6 2022-02-25 澎湃新闻 【#美国会有意援助乌克兰6亿美元武器#，帮助该国“打赢自己的战争”】 4740735829809929
7 2022-02-25 澎湃新闻 【#乌克兰民众排队献血#：不过举手之劳，怎能无动于衷】 4740747229660706
8 2022-02-25 澎湃新闻 【正在直播丨#俄战斗机群逼近乌首都基辅#，#连线基辅避难所中国留学生#】 4740771506290693
9 2022-02-25 澎湃新闻 【微博：发布“乌克兰美女来中国”等恶俗言论，74个账号被禁言】 4740818977423519
10 2022-02-25 澎湃新闻 #乌克兰承认基辅被击落的是己方战机#【学者圆桌：#俄罗斯闪击乌克兰只是危机序幕#？】 4740838208835952
11 2022-02-26 澎湃新闻 【#乌总统泽连斯基深夜街头露面#：目前仍在基辅】 4741031327171856
12 2022-02-26 澎湃新闻 【外媒：#27国同意向乌克兰提供更多军援#，包括弹药、武器等】 4741174281373011
13 2022-02-26 澎湃新闻 #乌总统称俄方毫无胜算#【基辅街头夜间猛烈交火，中国驻乌大使：不可能抛下同胞】 4741239792992749
14 2022-02-27 澎湃新闻 【法媒：#近万法军进入战备状态#，将派1500人加强北

In [ ]:
main()

15 2022-02-27 澎湃新闻 【对话在乌中国人：被炮火笼罩的恐惧与乡愁】 4741592772514063
16 2022-03-01 澎湃新闻 【中国驻乌克兰大使馆：#在乌中国公民可以乘乌专列撤往西部邻国#】 4742135851781696
17 2022-02-28 新华社 【#外交部介绍在乌中国公民撤离进展#】外交部发言人汪文斌在2月28日举行的外交部例行记者会上应询介绍了在乌克兰中国公民撤离的进展情况。 4741890563638622
18 2022-02-27 新华社 【俄国防部：#俄乌军事人员共同保障切尔诺贝利核电站安全#】 4741399364766388
19 2022-02-25 新华社 【#新华社记者逆行奔赴基辅#】 4740788883296224
20 2022-02-24 新华社 【#普京和卢卡申科通电话谈顿巴斯地区局势#】 4740497413245609
21 2022-02-24 新华社 #中方反对任何煽动战争的行为#【#华春莹说美国一直在煽动战争危险#】 4740446821550806
22 2022-02-24 新华社 新华社快讯：24日清晨，#乌克兰首都基辅上空传来数次爆炸声#。  4740365510512891
23 2022-03-02 凤凰网 【#乌克兰网球女将击败俄罗斯选手# 赛后承诺将奖金捐给乌克兰军队】 4742541608489625
24 2022-03-01 凤凰网 【#乌克兰将对助乌作战的外籍人士实行免签# 】 4742144013633024
25 2022-03-01 凤凰网 【#世界跆拳道联盟撤销普京黑带九段荣誉称号# 】 4742132013465801
26 2022-02-28 凤凰网 【乌克兰媒体称俄乌谈判已结束 俄媒：还在进行中】 4741966115900771
27 2022-02-28 凤凰网 【#俄战略导弹部队和北方舰队进入战备状态# 】 4741933831819293
28 2022-02-28 凤凰网 【#泽连斯基宣布释放有军事经验的罪犯参战#】 4741902643760675
29 2022-02-28 凤凰网 【民调：乌克兰总统#泽连斯基支持率飙升至90%以上#】 4741807717748671
30 2022-02-28 凤凰网 【#普京在什么情况下会动

In [ ]:
datas = []
mid=str(4740666338574356)
get_comment_index(mid,datas)

In [ ]:
pd.DataFrame(datas)

,user_name,commtent,data_time,mid
0,小愚-MR,乌克兰看好了，你的所谓盟友是怎么挣钱的,Fri Feb 25 07:53:26 +0800 2022,4740668062960982
1,Gemini_西_,这是开始明抢了,Fri Feb 25 07:47:46 +0800 2022,4740666637420446
2,余文浩先森,拜登：反正我已经拱完火了，我要开始背后敛财了。你们打归打，闹归闹，黄金美刀我都要,Fri Feb 25 07:51:10 +0800 2022,4740667492534246
3,你们的美少女哪吒,和国境线安全比起来这算个屁啊,Fri Feb 25 07:47:52 +0800 2022,4740666662062482
4,船长人生之星辰大海,老美牛逼，明抢,Fri Feb 25 07:51:54 +0800 2022,4740667677608174
...,...,...,...,...
62,牛若丸大叔,抢完阿富汗抢俄罗斯了真不怕打大的啊,Fri Feb 25 08:41:19 +0800 2022,4740680117125186
63,每天都想冬眠,无耻，坑乌克兰和俄罗斯打仗自己躲背后赚钱,Fri Feb 25 08:06:51 +0800 2022,4740671439899675
64,古玉和紫砂艺术研究,美欧的目的就是逼饿动手，然后明抢！,Fri Feb 25 08:03:58 +0800 2022,4740670714282618
65,村西来的狗蛋,前几年普京就说过，如果世界不需要俄罗斯了，那么这个世界也就没必要存在了！美国这么搞，真把俄罗...,Fri Feb 25 07:59:43 +0800 2022,4740669644736329


In [ ]:
l1 = [1,2,3]
l3 = [2,3,4]
l2=[]
l2.extend(l1)
l2.extend(l3)
l2

[1, 2, 3, 2, 3, 4]

In [ ]:
pd.DataFrame(l2)

,0,1,2
0,1,2,3
1,2,3,4


In [ ]:
datas = []
mid=str(4740677163814177)
get_comment_index(mid)

In [ ]:
pd.DataFrame(datas)

,user_name,commtent,data_time,mid
0,九洲大道中,北约：今天帮你，战争今天就上门,Fri Feb 25 08:30:14 +0800 2022,4740677323196641
1,麦晓慕,今天帮了乌克兰，今天就有战争上门,Fri Feb 25 08:30:09 +0800 2022,4740677302224943
2,庐山防滑链,悲剧，自己国家的安全需要别人来帮忙。把自己命运交给西方嘴皮子政客。兄弟，你把剧本拿错了。,Fri Feb 25 08:32:26 +0800 2022,4740677877109182
3,MurasameZero,欧洲:本来你没了就结束的东西，你还想拖整个欧洲下水啊,Fri Feb 25 08:31:11 +0800 2022,4740677562797652
4,CXN猫LN,帮了就不单单两个国家的战争啦,Fri Feb 25 08:32:49 +0800 2022,4740677974100643
...,...,...,...,...
191,文文1970X,你觉得能唬住他们？,Fri Feb 25 09:15:34 +0800 2022,4740688733017813
192,墨西哥祁明,,Fri Feb 25 09:08:19 +0800 2022,4740686912422952
193,千年红杏儿,刚看到一个新闻，说乌克兰没有向北约提出兵要求,Fri Feb 25 09:07:33 +0800 2022,4740686719746390
194,R-Augustus,北约: 一定支持你，加油，再送你2000个头盔,Fri Feb 25 08:49:32 +0800 2022,4740682180463366


In [ ]:
url_to_mid('Lh4ckg0eZ')

Ze0gkc4hL
3


4740677163814177